# Longitudinal RSA Analysis: VOTC Category Selectivity

**Purpose**: Analyze representational geometry stability in ventral occipitotemporal cortex (VOTC) following pediatric resection.

**Key Measures**:
- Geometry Preservation: RDM correlation between T1 and T2 (higher = more stable)
- Distinctiveness: Mean correlation between preferred and non-preferred categories (lower = more selective)

**Methodology** (based on Kriegeskorte et al. 2008; Nili et al. 2014; Liu et al. 2018):
1. Extract category-selective ROIs using top-20% voxel thresholding
2. Build RDM using 1 - Pearson correlation of beta patterns
3. Compare RDMs across sessions using Pearson correlation of vectorized upper triangles

**Critical Note**: ROI extraction and pattern extraction must use CONSISTENT contrast maps.

In [1]:
# CELL 1: Setup and Configuration
# ================================

import pandas as pd
import numpy as np
import nibabel as nib
from pathlib import Path
from scipy.ndimage import label, center_of_mass
from scipy.stats import pearsonr, spearmanr, ttest_ind
from scipy.spatial.distance import squareform
import warnings
warnings.filterwarnings('ignore')

# === FILE PATHS ===
CSV_FILE = Path('/user_data/csimmon2/git_repos/long_pt/long_pt_sub_info.csv')
BASE_DIR = Path('/user_data/csimmon2/long_pt')

# === SESSION START OVERRIDES ===
# Some subjects have unusable early sessions
SESSION_START = {'sub-010': 2, 'sub-018': 2, 'sub-068': 2}

# === CONTRAST MAPS ===
# These define which cope files to use for each category
# CRITICAL: Use the SAME map for ROI extraction AND pattern extraction

COPE_MAP_DIFFERENTIAL = {
    # Category > Other categories (differential contrasts)
    'face': (10, 1),    # Face > Objects (cope10)
    'word': (13, -1),   # Word > Scrambled (cope13, sign-flipped to match direction)
    'object': (3, 1),   # Object > Scrambled (cope3)
    'house': (11, 1)    # House > Objects (cope11)
}

COPE_MAP_SCRAMBLE = {
    # Category > Scrambled baseline (simpler contrasts)
    'face': (10, 1),    # Face > Scrambled
    'word': (12, 1),    # Word > Scrambled
    'object': (3, 1),   # Object > Scrambled
    'house': (11, 1)    # House > Scrambled
}

# === ANALYSIS PARAMETERS ===
CATEGORIES = ['face', 'word', 'object', 'house']
SPHERE_RADIUS = 6  # mm, for pattern extraction
ROI_PERCENTILE = 80  # Top 20% of voxels
MIN_CLUSTER_SIZE = 20  # Minimum voxels for valid ROI
MIN_Z_THRESHOLD = 1.64  # Floor for adaptive threshold (~p<0.05 one-tailed)

print("✓ Cell 1: Configuration loaded")

✓ Cell 1: Configuration loaded


In [2]:
# CELL 2: Load Subject Information
# =================================

def load_subjects():
    """Load all subjects from CSV and organize by group."""
    df = pd.read_csv(CSV_FILE)
    subjects = {}
    
    for _, row in df.iterrows():
        sid = row['sub']
        subj_dir = BASE_DIR / sid
        if not subj_dir.exists():
            continue
        
        # Get available sessions
        sessions = sorted(
            [d.name.replace('ses-', '') for d in subj_dir.glob('ses-*') if d.is_dir()],
            key=int
        )
        
        # Apply session start override
        start = SESSION_START.get(sid, 1)
        sessions = [s for s in sessions if int(s) >= start]
        if not sessions:
            continue
        
        # Determine hemisphere (for patients: intact hemisphere)
        is_patient = row.get('patient', 0) == 1
        if is_patient:
            intact_hemi = row.get('intact_hemi', 'left')
            hemi = 'l' if intact_hemi == 'left' else 'r'
        else:
            hemi = 'l'  # Default for controls (will extract both anyway)
        
        # Determine group
        if is_patient:
            group = row.get('group', 'unknown')
        else:
            group = 'control'
        
        subjects[sid] = {
            'sessions': sessions,
            'hemi': hemi,
            'group': group,
            'is_patient': is_patient
        }
    
    return subjects

SUBJECTS = load_subjects()

# Print summary
print(f"✓ Loaded {len(SUBJECTS)} subjects")
for g in ['OTC', 'nonOTC', 'control']:
    n = sum(1 for v in SUBJECTS.values() if v['group'] == g)
    print(f"  {g}: {n}")

✓ Loaded 25 subjects
  OTC: 7
  nonOTC: 9
  control: 9


In [3]:
# CELL 3: Core Functions
# ======================

def create_sphere(center_mni, affine, shape, radius=6):
    """
    Create a spherical mask centered at MNI coordinates.
    
    Parameters:
        center_mni: [x, y, z] in MNI/world coordinates
        affine: 4x4 affine matrix from reference image
        shape: (nx, ny, nz) shape of brain volume
        radius: sphere radius in mm
    
    Returns:
        3D boolean mask
    """
    # Create grid of voxel coordinates
    i, j, k = np.meshgrid(
        np.arange(shape[0]),
        np.arange(shape[1]),
        np.arange(shape[2]),
        indexing='ij'
    )
    voxel_coords = np.stack([i, j, k], axis=-1).reshape(-1, 3)
    
    # Convert to world coordinates
    world_coords = nib.affines.apply_affine(affine, voxel_coords)
    
    # Calculate distances from center
    distances = np.linalg.norm(world_coords - center_mni, axis=1)
    
    # Create mask
    mask = (distances <= radius).reshape(shape)
    return mask


def extract_top20_rois(cope_map, percentile=80, min_cluster=20, min_z=1.64):
    """
    Extract ROIs using top-N% voxel thresholding.
    
    Methodology (following Liu et al. 2018):
    1. Load z-stat map within category-specific search mask
    2. Compute adaptive threshold: max(percentile of positive voxels, min_z)
    3. Find largest suprathreshold cluster
    4. Return centroid and metadata
    
    Parameters:
        cope_map: dict mapping category -> (cope_num, multiplier)
        percentile: percentile threshold (80 = top 20%)
        min_cluster: minimum voxels for valid cluster
        min_z: floor for adaptive threshold
    
    Returns:
        dict: {subject_id: {roi_key: {session: {centroid, n_voxels, threshold, ...}}}}
    """
    all_rois = {}
    
    for sid, info in SUBJECTS.items():
        first_ses = info['sessions'][0]
        roi_dir = BASE_DIR / sid / f'ses-{first_ses}' / 'ROIs'
        if not roi_dir.exists():
            continue
        
        subject_rois = {}
        
        # Extract BOTH hemispheres for all subjects
        for hemi in ['l', 'r']:
            for cat, (cope_num, mult) in cope_map.items():
                
                # Load search mask
                mask_file = roi_dir / f'{hemi}_{cat}_searchmask.nii.gz'
                if not mask_file.exists():
                    continue
                
                try:
                    mask_img = nib.load(mask_file)
                    search_mask = mask_img.get_fdata() > 0
                    affine = mask_img.affine
                    shape = mask_img.shape
                except:
                    continue
                
                roi_key = f'{hemi}_{cat}'
                subject_rois[roi_key] = {}
                
                # Process each session
                for ses in info['sessions']:
                    feat_dir = BASE_DIR / sid / f'ses-{ses}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat'
                    
                    # Handle registration: first session uses native, others registered
                    if ses == first_ses:
                        z_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / 'zstat1.nii.gz'
                    else:
                        z_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / f'zstat1_ses{first_ses}.nii.gz'
                    
                    if not z_file.exists():
                        continue
                    
                    try:
                        z_data = nib.load(z_file).get_fdata() * mult
                        
                        # Get positive voxels within search mask
                        pos_mask = search_mask & (z_data > 0)
                        pos_voxels = z_data[pos_mask]
                        
                        if len(pos_voxels) < min_cluster:
                            continue
                        
                        # Adaptive threshold: top N% or minimum z
                        dynamic_thresh = max(np.percentile(pos_voxels, percentile), min_z)
                        
                        # Find suprathreshold voxels
                        suprathresh = (z_data > dynamic_thresh) & search_mask
                        labeled, n_clusters = label(suprathresh)
                        
                        if n_clusters == 0:
                            continue
                        
                        # Find largest cluster
                        cluster_sizes = [(i, np.sum(labeled == i)) for i in range(1, n_clusters + 1)]
                        best_idx, best_size = max(cluster_sizes, key=lambda x: x[1])
                        
                        if best_size < min_cluster:
                            continue
                        
                        # Get cluster mask and centroid
                        cluster_mask = (labeled == best_idx)
                        centroid_vox = center_of_mass(cluster_mask)
                        centroid_mni = nib.affines.apply_affine(affine, centroid_vox)
                        
                        # Get peak z-value
                        peak_idx = np.unravel_index(np.argmax(z_data * cluster_mask), shape)
                        peak_z = z_data[peak_idx]
                        
                        subject_rois[roi_key][ses] = {
                            'centroid': centroid_mni,
                            'n_voxels': int(best_size),
                            'peak_z': float(peak_z),
                            'threshold': float(dynamic_thresh),
                            'affine': affine,
                            'shape': shape
                        }
                    except Exception as e:
                        continue
        
        if subject_rois:
            all_rois[sid] = subject_rois
    
    return all_rois


def compute_rdm(patterns, method='correlation'):
    """
    Compute representational dissimilarity matrix.
    
    Parameters:
        patterns: list of 1D arrays (one per category)
        method: 'correlation' (1 - r) or 'euclidean'
    
    Returns:
        n x n dissimilarity matrix
    """
    n = len(patterns)
    rdm = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            if i == j:
                rdm[i, j] = 0
            elif method == 'correlation':
                r, _ = pearsonr(patterns[i], patterns[j])
                rdm[i, j] = 1 - r
            else:  # euclidean
                rdm[i, j] = np.linalg.norm(patterns[i] - patterns[j])
    
    return rdm


print("✓ Cell 3: Core functions defined")

✓ Cell 3: Core functions defined


In [4]:
# CELL 4: Extract ROIs
# ====================

print("Extracting ROIs with top-20% threshold...")
print("Using COPE_MAP_DIFFERENTIAL (matches Liu et al. 2018 approach)\n")

rois = extract_top20_rois(
    COPE_MAP_DIFFERENTIAL,
    percentile=ROI_PERCENTILE,
    min_cluster=MIN_CLUSTER_SIZE,
    min_z=MIN_Z_THRESHOLD
)

print(f"✓ Extracted ROIs for {len(rois)} subjects")

# Diagnostic: Show sample thresholds
print("\nSample thresholds (verifying adaptive method):")
count = 0
for sid in list(rois.keys())[:5]:
    for roi_key, sessions in rois[sid].items():
        for ses, data in sessions.items():
            print(f"  {sid} {roi_key} ses-{ses}: z>{data['threshold']:.2f}, n={data['n_voxels']}")
            count += 1
            if count >= 5:
                break
        if count >= 5:
            break
    if count >= 5:
        break

Extracting ROIs with top-20% threshold...
Using COPE_MAP_DIFFERENTIAL (matches Liu et al. 2018 approach)

✓ Extracted ROIs for 24 subjects

Sample thresholds (verifying adaptive method):
  sub-004 l_face ses-01: z>1.66, n=846
  sub-004 l_face ses-02: z>3.27, n=1591
  sub-004 l_face ses-03: z>3.67, n=2062
  sub-004 l_face ses-05: z>4.04, n=1761
  sub-004 l_face ses-06: z>4.32, n=1953


In [5]:
# CELL 5: Geometry Preservation Analysis
# ======================================
# 
# Methodology:
# 1. For each ROI, create 6mm sphere at centroid
# 2. Extract beta patterns for all 4 categories
# 3. Compute 4x4 RDM (1 - correlation)
# 4. Correlate RDM upper triangle between first and last session
# 
# Interpretation:
# - High correlation (near 1): Stable representational geometry
# - Low correlation (near 0): Representational reorganization

def compute_geometry_preservation(rois_dict, cope_map, radius=6):
    """
    Compute geometry preservation: RDM stability across sessions.
    
    CRITICAL: cope_map must match the one used for ROI extraction!
    """
    results = []
    
    for sid, roi_data in rois_dict.items():
        info = SUBJECTS.get(sid)
        if not info:
            continue
        
        first_ses = info['sessions'][0]
        
        # Get reference image for affine/shape
        roi_dir = BASE_DIR / sid / f'ses-{first_ses}' / 'ROIs'
        ref_file = None
        for cat in CATEGORIES:
            for h in ['l', 'r']:
                test = roi_dir / f'{h}_{cat}_searchmask.nii.gz'
                if test.exists():
                    ref_file = test
                    break
            if ref_file:
                break
        
        if not ref_file:
            continue
        
        ref_img = nib.load(ref_file)
        affine = ref_img.affine
        shape = ref_img.shape
        
        for roi_key, sessions_data in roi_data.items():
            sessions = sorted(sessions_data.keys())
            if len(sessions) < 2:
                continue
            
            first_s, last_s = sessions[0], sessions[-1]
            
            # Create spheres at each session's centroid
            sphere_t1 = create_sphere(sessions_data[first_s]['centroid'], affine, shape, radius)
            sphere_t2 = create_sphere(sessions_data[last_s]['centroid'], affine, shape, radius)
            
            rdms = {}
            
            for ses, sphere in [(first_s, sphere_t1), (last_s, sphere_t2)]:
                feat_dir = BASE_DIR / sid / f'ses-{ses}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat'
                
                patterns = []
                valid = True
                
                for cat in CATEGORIES:
                    cope_num, mult = cope_map[cat]
                    
                    # Use z-stats for pattern (consistent with ROI extraction)
                    if ses == first_ses:
                        cope_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / 'zstat1.nii.gz'
                    else:
                        cope_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / f'zstat1_ses{first_ses}.nii.gz'
                    
                    if not cope_file.exists():
                        valid = False
                        break
                    
                    data = nib.load(cope_file).get_fdata() * mult
                    pattern = data[sphere]
                    
                    if len(pattern) == 0 or not np.all(np.isfinite(pattern)):
                        valid = False
                        break
                    
                    patterns.append(pattern)
                
                if not valid or len(patterns) != 4:
                    continue
                
                try:
                    rdm = compute_rdm(patterns, method='correlation')
                    rdms[ses] = rdm
                except:
                    continue
            
            # Compute geometry preservation if we have both RDMs
            if len(rdms) == 2:
                # Extract upper triangle (6 unique pairwise distances)
                triu_idx = np.triu_indices(4, k=1)
                vec_t1 = rdms[first_s][triu_idx]
                vec_t2 = rdms[last_s][triu_idx]
                
                # Pearson correlation of RDM vectors
                r, p = pearsonr(vec_t1, vec_t2)
                
                hemi = roi_key.split('_')[0]
                cat = roi_key.split('_')[1]
                
                results.append({
                    'subject': sid,
                    'group': info['group'],
                    'roi': roi_key,
                    'hemi': hemi,
                    'category': cat,
                    'category_type': 'Bilateral' if cat in ['object', 'house'] else 'Unilateral',
                    'geometry_preservation': r,
                    'p_value': p,
                    'n_sessions': len(sessions),
                    'first_ses': first_s,
                    'last_ses': last_s
                })
    
    return pd.DataFrame(results)


# Run analysis
print("Computing Geometry Preservation...")
print(f"Using {SPHERE_RADIUS}mm spheres, COPE_MAP_DIFFERENTIAL\n")

geometry_df = compute_geometry_preservation(rois, COPE_MAP_DIFFERENTIAL, radius=SPHERE_RADIUS)

print(f"✓ Computed {len(geometry_df)} ROI measurements")

Computing Geometry Preservation...
Using 6mm spheres, COPE_MAP_DIFFERENTIAL

✓ Computed 132 ROI measurements


In [6]:
# CELL 6: Geometry Preservation Results
# =====================================

if len(geometry_df) > 0:
    print("="*60)
    print("GEOMETRY PRESERVATION RESULTS")
    print("Higher values = more stable representational geometry")
    print("="*60)
    
    # Summary by Group and Category Type
    print("\n1. Summary by Group and Category Type:")
    summary = geometry_df.groupby(['group', 'category_type'])['geometry_preservation'].agg(['mean', 'std', 'count'])
    print(summary.round(3))
    
    # Statistical tests: OTC Bilateral vs Unilateral
    print("\n2. Statistical Tests (OTC patients):")
    otc_data = geometry_df[geometry_df['group'] == 'OTC']
    if len(otc_data) > 0:
        bil = otc_data[otc_data['category_type'] == 'Bilateral']['geometry_preservation']
        uni = otc_data[otc_data['category_type'] == 'Unilateral']['geometry_preservation']
        
        if len(bil) > 1 and len(uni) > 1:
            t, p = ttest_ind(bil, uni)
            print(f"   Bilateral: {bil.mean():.3f} ± {bil.std():.3f} (n={len(bil)})")
            print(f"   Unilateral: {uni.mean():.3f} ± {uni.std():.3f} (n={len(uni)})")
            print(f"   t-test: t={t:.3f}, p={p:.4f}")
            
            if p < 0.05:
                print("   ✓ SIGNIFICANT: Bilateral categories show different stability than unilateral")
    
    # Compare OTC vs nonOTC vs Controls
    print("\n3. Group Comparisons (Bilateral categories only):")
    bil_data = geometry_df[geometry_df['category_type'] == 'Bilateral']
    for g1, g2 in [('OTC', 'nonOTC'), ('OTC', 'control'), ('nonOTC', 'control')]:
        d1 = bil_data[bil_data['group'] == g1]['geometry_preservation']
        d2 = bil_data[bil_data['group'] == g2]['geometry_preservation']
        if len(d1) > 1 and len(d2) > 1:
            t, p = ttest_ind(d1, d2)
            print(f"   {g1} vs {g2}: t={t:.3f}, p={p:.4f}")
else:
    print("No geometry results computed. Check ROI extraction.")

GEOMETRY PRESERVATION RESULTS
Higher values = more stable representational geometry

1. Summary by Group and Category Type:
                        mean    std  count
group   category_type                     
OTC     Bilateral      0.424  0.349     12
        Unilateral     0.713  0.197     12
control Bilateral      0.659  0.359     36
        Unilateral     0.755  0.213     36
nonOTC  Bilateral      0.726  0.257     18
        Unilateral     0.762  0.196     18

2. Statistical Tests (OTC patients):
   Bilateral: 0.424 ± 0.349 (n=12)
   Unilateral: 0.713 ± 0.197 (n=12)
   t-test: t=-2.496, p=0.0205
   ✓ SIGNIFICANT: Bilateral categories show different stability than unilateral

3. Group Comparisons (Bilateral categories only):
   OTC vs nonOTC: t=-2.733, p=0.0107
   OTC vs control: t=-1.982, p=0.0534
   nonOTC vs control: t=0.700, p=0.4869


In [7]:
# CELL 7: Distinctiveness Analysis
# =================================
#
# Liu et al. (2018) Distinctiveness metric:
# Mean Fisher-transformed correlation between preferred and non-preferred categories
# Lower values = more distinctive/selective representations

def compute_distinctiveness(rois_dict, cope_map, radius=6):
    """
    Compute Liu distinctiveness: mean correlation with non-preferred categories.
    """
    roi_preferred = {
        'l_face': 'face', 'r_face': 'face',
        'l_word': 'word', 'r_word': 'word',
        'l_object': 'object', 'r_object': 'object',
        'l_house': 'house', 'r_house': 'house'
    }
    
    results = []
    
    for sid, roi_data in rois_dict.items():
        info = SUBJECTS.get(sid)
        if not info:
            continue
        
        first_ses = info['sessions'][0]
        
        # Get reference image
        roi_dir = BASE_DIR / sid / f'ses-{first_ses}' / 'ROIs'
        ref_file = None
        for cat in CATEGORIES:
            for h in ['l', 'r']:
                test = roi_dir / f'{h}_{cat}_searchmask.nii.gz'
                if test.exists():
                    ref_file = test
                    break
            if ref_file:
                break
        
        if not ref_file:
            continue
        
        ref_img = nib.load(ref_file)
        affine = ref_img.affine
        shape = ref_img.shape
        
        for roi_key, sessions_data in roi_data.items():
            if roi_key not in roi_preferred:
                continue
            
            preferred_cat = roi_preferred[roi_key]
            pref_idx = CATEGORIES.index(preferred_cat)
            nonpref_indices = [i for i, c in enumerate(CATEGORIES) if c != preferred_cat]
            
            for ses, ses_data in sessions_data.items():
                sphere = create_sphere(ses_data['centroid'], affine, shape, radius)
                feat_dir = BASE_DIR / sid / f'ses-{ses}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat'
                
                patterns = []
                valid = True
                
                for cat in CATEGORIES:
                    cope_num, mult = cope_map[cat]
                    
                    if ses == first_ses:
                        cope_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / 'zstat1.nii.gz'
                    else:
                        cope_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / f'zstat1_ses{first_ses}.nii.gz'
                    
                    if not cope_file.exists():
                        valid = False
                        break
                    
                    data = nib.load(cope_file).get_fdata() * mult
                    pattern = data[sphere]
                    
                    if len(pattern) == 0 or not np.all(np.isfinite(pattern)):
                        valid = False
                        break
                    
                    patterns.append(pattern)
                
                if not valid or len(patterns) != 4:
                    continue
                
                # Compute correlation matrix
                try:
                    corr_matrix = np.corrcoef(patterns)
                    
                    # Fisher transform correlations
                    corr_fisher = np.arctanh(np.clip(corr_matrix, -0.999, 0.999))
                    
                    # Mean correlation with non-preferred categories
                    pref_vs_nonpref = [corr_fisher[pref_idx, i] for i in nonpref_indices]
                    distinctiveness = np.mean(pref_vs_nonpref)
                    
                    hemi = roi_key.split('_')[0]
                    
                    results.append({
                        'subject': sid,
                        'group': info['group'],
                        'roi': roi_key,
                        'hemi': hemi,
                        'category': preferred_cat,
                        'session': ses,
                        'distinctiveness': distinctiveness
                    })
                except:
                    continue
    
    return pd.DataFrame(results)


print("Computing Distinctiveness...")
distinctiveness_df = compute_distinctiveness(rois, COPE_MAP_DIFFERENTIAL, radius=SPHERE_RADIUS)
print(f"✓ Computed {len(distinctiveness_df)} measurements")

Computing Distinctiveness...
✓ Computed 294 measurements


In [8]:
# CELL 8: Distinctiveness Results
# ================================

if len(distinctiveness_df) > 0:
    print("="*60)
    print("DISTINCTIVENESS RESULTS")
    print("Lower values = more selective/distinctive representations")
    print("="*60)
    
    # Average across sessions per subject/ROI
    avg_dist = distinctiveness_df.groupby(['subject', 'group', 'roi', 'category'])['distinctiveness'].mean().reset_index()
    
    # Summary by group and category
    print("\nMean Distinctiveness by Group and Category:")
    summary = avg_dist.groupby(['group', 'category'])['distinctiveness'].agg(['mean', 'std', 'count'])
    print(summary.round(3))
else:
    print("No distinctiveness results computed.")

DISTINCTIVENESS RESULTS
Lower values = more selective/distinctive representations

Mean Distinctiveness by Group and Category:
                   mean    std  count
group   category                     
OTC     face      0.099  0.252      6
        house     0.298  0.126      6
        object    0.263  0.220      6
        word     -0.081  0.161      6
control face      0.225  0.196     18
        house     0.377  0.197     18
        object    0.351  0.217     18
        word     -0.020  0.223     18
nonOTC  face     -0.039  0.146      9
        house     0.350  0.164      9
        object    0.192  0.267      9
        word     -0.129  0.085      9


In [9]:
# CELL 9: Diagnostic - Compare Contrast Maps
# ==========================================
#
# This cell verifies that DIFFERENTIAL and SCRAMBLE produce valid but different RDMs

print("DIAGNOSTIC: Comparing contrast map effects on RDM\n")

# Pick first subject with ROIs
test_sid = list(rois.keys())[0]
test_info = SUBJECTS[test_sid]
first_ses = test_info['sessions'][0]

# Get a sphere location
roi_key = list(rois[test_sid].keys())[0]
centroid = rois[test_sid][roi_key][first_ses]['centroid']

# Load reference
roi_dir = BASE_DIR / test_sid / f'ses-{first_ses}' / 'ROIs'
ref_file = roi_dir / f'{roi_key.split("_")[0]}_face_searchmask.nii.gz'
ref_img = nib.load(ref_file)
affine = ref_img.affine
shape = ref_img.shape

sphere = create_sphere(centroid, affine, shape, radius=SPHERE_RADIUS)
feat_dir = BASE_DIR / test_sid / f'ses-{first_ses}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat'

print(f"Subject: {test_sid}, ROI: {roi_key}")
print(f"Sphere voxels: {sphere.sum()}\n")

for map_name, cmap in [('DIFFERENTIAL', COPE_MAP_DIFFERENTIAL), ('SCRAMBLE', COPE_MAP_SCRAMBLE)]:
    patterns = []
    print(f"{map_name}:")
    
    for cat in CATEGORIES:
        cope_num, mult = cmap[cat]
        cope_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / 'zstat1.nii.gz'
        
        if cope_file.exists():
            data = nib.load(cope_file).get_fdata() * mult
            pattern = data[sphere]
            patterns.append(pattern)
            print(f"  {cat}: cope{cope_num}x{mult:+d}, mean={pattern.mean():.2f}, std={pattern.std():.2f}")
    
    if len(patterns) == 4:
        rdm = compute_rdm(patterns)
        triu = rdm[np.triu_indices(4, k=1)]
        print(f"  RDM upper triangle: {np.round(triu, 3)}")
        print(f"  RDM range: [{triu.min():.3f}, {triu.max():.3f}]")
        
        # Check for invalid values (should be 0-2 for correlation distance)
        if triu.max() > 2.0:
            print(f"  ⚠ WARNING: RDM values > 2 indicate negative correlations")
    print()

DIAGNOSTIC: Comparing contrast map effects on RDM

Subject: sub-004, ROI: l_face
Sphere voxels: 896

DIFFERENTIAL:
  face: cope10x+1, mean=1.73, std=0.81
  word: cope13x-1, mean=-1.40, std=0.75
  object: cope3x+1, mean=1.54, std=0.92
  house: cope11x+1, mean=0.69, std=0.72
  RDM upper triangle: [1.209 1.19  0.863 1.45  1.488 0.749]
  RDM range: [0.749, 1.488]

SCRAMBLE:
  face: cope10x+1, mean=1.73, std=0.81
  word: cope12x+1, mean=-0.09, std=1.01
  object: cope3x+1, mean=1.54, std=0.92
  house: cope11x+1, mean=0.69, std=0.72
  RDM upper triangle: [1.187 1.19  0.863 0.711 0.708 0.749]
  RDM range: [0.708, 1.190]



In [10]:
# CELL 10: Save Results
# =====================

output_dir = Path('/user_data/csimmon2/git_repos/long_pt/B_analyses')
output_dir.mkdir(exist_ok=True, parents=True)

# Save geometry preservation
if len(geometry_df) > 0:
    geom_file = output_dir / 'geometry_preservation_differential.csv'
    geometry_df.to_csv(geom_file, index=False)
    print(f"✓ Saved geometry results to {geom_file}")

# Save distinctiveness
if len(distinctiveness_df) > 0:
    dist_file = output_dir / 'distinctiveness_differential.csv'
    distinctiveness_df.to_csv(dist_file, index=False)
    print(f"✓ Saved distinctiveness results to {dist_file}")

print("\n" + "="*60)
print("ANALYSIS COMPLETE")
print("="*60)

✓ Saved geometry results to /user_data/csimmon2/git_repos/long_pt/B_analyses/geometry_preservation_differential.csv
✓ Saved distinctiveness results to /user_data/csimmon2/git_repos/long_pt/B_analyses/distinctiveness_differential.csv

ANALYSIS COMPLETE
